# RFID:

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import colormaps as cmaps
from matplotlib.patches import Circle
import math
import random
import copy 
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from numba import njit
from numba.typed import List
import networkx as nx


## Các hàm sử dụng

### Fuzzy Logic Controller (Type 1 Madani model) 

In [ ]:
def switchProtocol(empty, remain, collision):
    """Dùng FLC để quyết định chuyển protocol dựa trên các tham số:
    - empty: Tỷ lệ slot trống (0-1)
    - remain: Tỷ lệ thẻ còn lại (0-1)
    - collision: Tỷ lệ slot va chạm (0-1)
    ->Trả về quyết định chuyển protocol: Switching hoặc Not Switching
    """
    # Khai báo biến mờ
    emptyRate = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'emptyRate')
    tagRemain = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'tagRemain')
    collisionRate = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'collisionRate')

    switchDecision = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'switchDecision')

    # Định nghĩa hàm thuộc
    emptyRate['low'] = fuzz.trimf(emptyRate.universe, [0, 0, 0.75])
    emptyRate['medium'] = fuzz.trimf(emptyRate.universe, [0.6, 0.63, 0.8])
    emptyRate['high'] = fuzz.trimf(emptyRate.universe, [0.7, 1, 1])

    tagRemain['low'] = fuzz.trimf(tagRemain.universe, [0, 0, 0.5])
    tagRemain['medium'] = fuzz.trimf(tagRemain.universe, [0.4, 0.6, 0.8])
    tagRemain['high'] = fuzz.trimf(tagRemain.universe, [0.7, 1, 1])

    collisionRate['low'] = fuzz.trimf(collisionRate.universe, [0, 0, 0.5])
    collisionRate['medium'] = fuzz.trimf(collisionRate.universe, [0.4, 0.6, 0.8])
    collisionRate['high'] = fuzz.trimf(collisionRate.universe, [0.7, 1, 1])

    switchDecision['not switching'] = fuzz.trimf(switchDecision.universe, [0, 0, 0.3])
    switchDecision['little consideration'] = fuzz.trimf(switchDecision.universe, [0.25, 0.35, 0.4])
    switchDecision['consideration'] = fuzz.trimf(switchDecision.universe, [0.4, 0.6, 0.7])
    switchDecision['should switching'] = fuzz.trimf(switchDecision.universe, [0.65, 0.8, 0.9])
    switchDecision['must switching'] = fuzz.trimf(switchDecision.universe, [0.85, 1, 1])

    # Tạo luật mờ
    rule1 = ctrl.Rule(emptyRate['low'] & tagRemain['high'] & collisionRate['low'], switchDecision['not switching'])
    rule2 = ctrl.Rule(emptyRate['low'] & tagRemain['high'] & collisionRate['medium'], switchDecision['not switching'])
    rule3 = ctrl.Rule(emptyRate['low'] & tagRemain['high'] & collisionRate['high'], switchDecision['not switching'])
    rule4 = ctrl.Rule(emptyRate['medium'] & tagRemain['high'] & collisionRate['low'], switchDecision['little consideration'])
    rule5 = ctrl.Rule(emptyRate['medium'] & tagRemain['high'] & collisionRate['medium'], switchDecision['little consideration'])
    rule6 = ctrl.Rule(emptyRate['medium'] & tagRemain['high'] & collisionRate['high'], switchDecision['not switching'])
    rule7 = ctrl.Rule(emptyRate['high'] & tagRemain['high'] & collisionRate['low'], switchDecision['consideration'])
    rule8 = ctrl.Rule(emptyRate['high'] & tagRemain['high'] & collisionRate['medium'], switchDecision['little consideration'])
    rule9 = ctrl.Rule(emptyRate['high'] & tagRemain['high'] & collisionRate['high'], switchDecision['not switching'])

    rule10 = ctrl.Rule(emptyRate['low'] & tagRemain['medium'] & collisionRate['low'], switchDecision['consideration'])
    rule11 = ctrl.Rule(emptyRate['low'] & tagRemain['medium'] & collisionRate['medium'], switchDecision['little consideration'])
    rule12 = ctrl.Rule(emptyRate['low'] & tagRemain['medium'] & collisionRate['high'], switchDecision['not switching'])
    rule13 = ctrl.Rule(emptyRate['medium'] & tagRemain['medium'] & collisionRate['low'], switchDecision['should switching'])
    rule14 = ctrl.Rule(emptyRate['medium'] & tagRemain['medium'] & collisionRate['medium'], switchDecision['consideration'])
    rule15 = ctrl.Rule(emptyRate['medium'] & tagRemain['medium'] & collisionRate['high'], switchDecision['little consideration'])
    rule16 = ctrl.Rule(emptyRate['high'] & tagRemain['medium'] & collisionRate['low'], switchDecision['must switching'])
    rule17 = ctrl.Rule(emptyRate['high'] & tagRemain['medium'] & collisionRate['medium'], switchDecision['should switching'])
    rule18 = ctrl.Rule(emptyRate['high'] & tagRemain['medium'] & collisionRate['high'], switchDecision['little consideration'])

    rule19 = ctrl.Rule(emptyRate['low'] & tagRemain['low'] & collisionRate['low'], switchDecision['should switching'])
    rule20 = ctrl.Rule(emptyRate['low'] & tagRemain['low'] & collisionRate['medium'], switchDecision['consideration'])
    rule21 = ctrl.Rule(emptyRate['low'] & tagRemain['low'] & collisionRate['high'], switchDecision['consideration'])
    rule22 = ctrl.Rule(emptyRate['medium'] & tagRemain['low'] & collisionRate['low'], switchDecision['must switching'])
    rule23 = ctrl.Rule(emptyRate['medium'] & tagRemain['low'] & collisionRate['medium'], switchDecision['should switching'])
    rule24 = ctrl.Rule(emptyRate['medium'] & tagRemain['low'] & collisionRate['high'], switchDecision['should switching'])
    rule25 = ctrl.Rule(emptyRate['high'] & tagRemain['low'] & collisionRate['low'], switchDecision['must switching'])
    rule26 = ctrl.Rule(emptyRate['high'] & tagRemain['low'] & collisionRate['medium'], switchDecision['must switching'])
    rule27 = ctrl.Rule(emptyRate['high'] & tagRemain['low'] & collisionRate['high'], switchDecision['must switching'])    

    # Tạo hệ thống và mô phỏng
    system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9,
                                 rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18,
                                 rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27])
    sim = ctrl.ControlSystemSimulation(system)

    sim.input['emptyRate'] = empty
    sim.input['tagRemain'] = remain
    sim.input['collisionRate'] = collision

    sim.compute()
    decision_value = sim.output['switchDecision']

    if decision_value < 0.65:
        return "Not Switching"
    else:
        return "Switching"


In [ ]:
def classificationPower(tagCountNear, tagCountFar, readerDensityValue):
    """Dùng FLC để quyết định mức công suất yêu cầu dựa trên các tham số:
    - tagCountNear: Tỷ lệ thẻ gần (0-1)
    - tagCountFar: Tỷ lệ thẻ xa (0-1)
    - readerDensityValue: Mật độ đầu đọc (0-1)
    ->Trả về mức công suất yêu cầu: Low Power hoặc High Power
    """
    tagNear = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'tagNear')
    tagFar = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'tagFar')
    readerDensity = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'readerDensity')
    
    powerRequest = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'powerRequest')

    # Định nghĩa hàm thuộc
    tagNear['low'] = fuzz.trimf(tagNear.universe, [0, 0, 0.45])
    tagNear['medium'] = fuzz.trimf(tagNear.universe, [0.4, 0.6, 0.8])
    tagNear['high'] = fuzz.trimf(tagNear.universe, [0.7, 1, 1])

    tagFar['none'] = fuzz.trimf(tagFar.universe, [0, 0, 0])
    tagFar['medium'] = fuzz.trimf(tagFar.universe, [0, 0.4, 0.7])
    tagFar['high'] = fuzz.trimf(tagFar.universe, [0.6, 1, 1])

    readerDensity['low'] = fuzz.trimf(readerDensity.universe, [0, 0, 0.5])
    readerDensity['medium'] = fuzz.trimf(readerDensity.universe, [0.4, 0.6, 0.8])
    readerDensity['high'] = fuzz.trimf(readerDensity.universe, [0.7, 1, 1])

    powerRequest['low'] = fuzz.trimf(powerRequest.universe, [0, 0, 0.3])
    powerRequest['consideration'] = fuzz.trimf(powerRequest.universe, [0.25, 0.5, 0.75])
    powerRequest['high'] = fuzz.trimf(powerRequest.universe, [0.7, 1, 1])   

    # Tạo luật mờ
    rule1 = ctrl.Rule(readerDensity['low'] & tagNear['low'] & tagFar['none'], powerRequest['low'])
    rule2 = ctrl.Rule(readerDensity['low'] & tagNear['low'] & tagFar['medium'], powerRequest['high'])
    rule3 = ctrl.Rule(readerDensity['low'] & tagNear['low'] & tagFar['high'], powerRequest['high'])
    rule4 = ctrl.Rule(readerDensity['low'] & tagNear['medium'] & tagFar['none'], powerRequest['low'])
    rule5 = ctrl.Rule(readerDensity['low'] & tagNear['medium'] & tagFar['medium'], powerRequest['high'])
    rule6 = ctrl.Rule(readerDensity['low'] & tagNear['medium'] & tagFar['high'], powerRequest['high'])
    rule7 = ctrl.Rule(readerDensity['low'] & tagNear['high'] & tagFar['none'], powerRequest['low'])
    rule8 = ctrl.Rule(readerDensity['low'] & tagNear['high'] & tagFar['medium'], powerRequest['high'])
    rule9 = ctrl.Rule(readerDensity['low'] & tagNear['high'] & tagFar['high'], powerRequest['high'])

    rule10 = ctrl.Rule(readerDensity['medium'] & tagNear['low'] & tagFar['none'], powerRequest['low'])
    rule11 = ctrl.Rule(readerDensity['medium'] & tagNear['low'] & tagFar['medium'], powerRequest['consideration'])
    rule12 = ctrl.Rule(readerDensity['medium'] & tagNear['low'] & tagFar['high'], powerRequest['high'])
    rule13 = ctrl.Rule(readerDensity['medium'] & tagNear['medium'] & tagFar['none'], powerRequest['low'])
    rule14 = ctrl.Rule(readerDensity['medium'] & tagNear['medium'] & tagFar['medium'], powerRequest['consideration'])
    rule15 = ctrl.Rule(readerDensity['medium'] & tagNear['medium'] & tagFar['high'], powerRequest['high'])
    rule16 = ctrl.Rule(readerDensity['medium'] & tagNear['high'] & tagFar['none'], powerRequest['low'])
    rule17 = ctrl.Rule(readerDensity['medium'] & tagNear['high'] & tagFar['medium'], powerRequest['low'])
    rule18 = ctrl.Rule(readerDensity['medium'] & tagNear['high'] & tagFar['high'], powerRequest['consideration'])

    rule19 = ctrl.Rule(readerDensity['high'] & tagNear['low'] & tagFar['none'], powerRequest['low'])
    rule20 = ctrl.Rule(readerDensity['high'] & tagNear['low'] & tagFar['medium'], powerRequest['low'])
    rule21 = ctrl.Rule(readerDensity['high'] & tagNear['low'] & tagFar['high'], powerRequest['consideration'])
    rule22 = ctrl.Rule(readerDensity['high'] & tagNear['medium'] & tagFar['none'], powerRequest['low'])
    rule23 = ctrl.Rule(readerDensity['high'] & tagNear['medium'] & tagFar['medium'], powerRequest['low'])
    rule24 = ctrl.Rule(readerDensity['high'] & tagNear['medium'] & tagFar['high'], powerRequest['consideration'])
    rule25 = ctrl.Rule(readerDensity['high'] & tagNear['high'] & tagFar['none'], powerRequest['low'])
    rule26 = ctrl.Rule(readerDensity['high'] & tagNear['high'] & tagFar['medium'], powerRequest['low'])
    rule27 = ctrl.Rule(readerDensity['high'] & tagNear['high'] & tagFar['high'], powerRequest['low'])

    # Tạo hệ thống và mô phỏng
    system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, 
                                 rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18,
                                 rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27])
    sim = ctrl.ControlSystemSimulation(system)

    sim.input['tagNear'] = tagCountNear
    sim.input['tagFar'] = tagCountFar
    sim.input['readerDensity'] = readerDensityValue

    sim.compute()
    decision_value = sim.output['powerRequest']

    return decision_value
    


### Reader 

In [ ]:
def generate_random_reader_coords(n_readers, area_size, seed=None):
    """Sinh ngẫu nhiên toạ độ (x, y) cho các reader trong vùng [0, area_size]"""
    if seed is not None:
        np.random.seed(seed)

    # random số thực trong [0, area_size]
    coords = np.random.uniform(0, area_size, size=(n_readers, 2))
    return coords

In [ ]:
def do_interfere(idx1, idx2, coords, radius):
    """Trả về True nếu vùng phủ sóng của reader idx1 và idx2 giao nhau."""
    dist = math.dist(coords[idx1], coords[idx2])
    return dist < 2 * radius

def can_be_added(r_idx, current_group, coords, r_candidate):
    """
    Kiểm tra xem reader r_idx có thể thêm vào nhóm không,
    xét theo khoảng cách và bán kính tương ứng.
    """
    for (member_id, _, r_member) in current_group:
        # Tránh giao thoa
        cond1 = not do_interfere(r_idx, member_id, coords, (r_member + r_candidate) / 2)
        if not cond1:
            return False
    return True


### Tags

#### ID

In [ ]:
def generate_unique_ids(n):
    """Tạo ID ngẫu nhiên duy nhất cho mỗi tag."""
    used_ids = set()
    ids = []
    while len(ids) < n:
        new_id = random.getrandbits(96)
        if new_id not in used_ids:
            used_ids.add(new_id)
            ids.append(new_id)  
    return ids

#### Tọa độ

In [ ]:
# Hàm tạo tọa độ ngẫu nhiên cho các tag trong vùng phủ sóng của các reader
def generate_random_coords(num_tags, num_readers, reader_coords, radius, seed=None):
    if seed is not None:
        np.random.seed(seed)
        
    tag_coords = []
    while len(tag_coords) < num_tags:
        # Sinh tọa độ ngẫu nhiên trong vùng [0, area_size]
        x = np.random.uniform(0, AREA_SIZE)
        y = np.random.uniform(0, AREA_SIZE)
        
        # Kiểm tra xem tọa độ này có nằm trong vùng phủ sóng của bất kỳ reader nào không
        for r_idx in range(num_readers):
            dist = math.dist((x, y), reader_coords[r_idx])
            if dist <= radius:
                tag_coords.append((x, y))
                break  # Dừng kiểm tra nếu đã tìm thấy một reader phù hợp
                
    return np.array(tag_coords[:num_tags])


In [ ]:
def generate_homogeneous_ppp(area_size, lam, seed=None):
    """
    Sinh phân bố Poisson đồng nhất (Homogeneous PPP) trong vùng vuông [0, area_size]^2.
    """
    if seed is not None:
        np.random.seed(seed)

    area = area_size ** 2
    num_points = np.random.poisson(lam * area)

    x = np.random.uniform(0, area_size, num_points)
    y = np.random.uniform(0, area_size, num_points)
    return np.column_stack((x, y))


In [ ]:
def create_regions(area_size, n_regions, lam_range, seed=None):
    """Tạo danh sách các vùng có mật độ λ khác nhau"""
    if seed is not None:
        np.random.seed(seed)

    regions = []
    for _ in range(n_regions):
        x1, x2 = sorted(np.random.uniform(0, area_size, 2))
        y1, y2 = sorted(np.random.uniform(0, area_size, 2))
        lam = np.random.uniform(*lam_range)
        regions.append({
            "x_range": (x1, x2),
            "y_range": (y1, y2),
            "lam": lam
        })
    return regions

def generate_inhomogeneous_ppp(regions, seed=None):
    """Sinh phân bố Poisson không đồng nhất (Inhomogeneous PPP)
    với các mật độ λ_i khác nhau theo vùng.
"""
    if seed is not None:
        np.random.seed(seed)

    tag_coords = []
    for region in regions:
        xmin, xmax = region["x_range"]
        ymin, ymax = region["y_range"]
        area = (xmax - xmin) * (ymax - ymin)
        num_tags = np.random.poisson(region["lam"] * area)
        
        tag_x = np.random.uniform(xmin, xmax, num_tags)
        tag_y = np.random.uniform(ymin, ymax, num_tags)
        tag_coords.append(np.column_stack((tag_x, tag_y)))

    return np.vstack(tag_coords) if tag_coords else np.empty((0, 2))

In [ ]:
def confirm_tags_in_range(reader_idx, tags, reader_coords, radius):
    """Xác định danh sách tag trong vùng phủ sóng của một reader."""
    tags_list = []
    for tag in tags:
        if math.dist(reader_coords[reader_idx], tag['coords']) <= radius:
            tags_list.append(tag)
    return tags_list

def confirm_tags_in_group_range(group, tags):
    """Trả về danh sách tag trong vùng phủ sóng của nhóm reader."""
    group_tags = []
    for reader_info in group:
        radius = reader_info[2]
        coord = reader_info[1]
        
        for tag in tags:
            if math.dist(coord, tag['coords']) <= radius:
                if tag not in group_tags:
                    group_tags.append(tag)
    return group_tags

#### Danh sách tag

In [ ]:
# # Hàm tạo danh sách các tag với ID và tọa độ ngẫu nhiên trong vùng phủ sóng của các reader
# def create_tags(num_tags, reader_coords, radius):
#     unique_ids = generate_unique_ids(num_tags)
#     tag_coords = generate_random_coords(num_tags, len(reader_coords), reader_coords, radius)

#     tags = [{
#         "id": uid, 
#         "coords": (float(coord[0]), float(coord[1])), 
#         "status": False
#     } for uid, coord in zip(unique_ids, tag_coords)]
#     return tags


In [ ]:
def create_tags_inhomo(regions):
    """Tạo danh sách các tag với ID và tọa độ ngẫu nhiên trong vùng [0, area_size] theo Inhomogeneous Poisson Point Process"""
    # tag_coords = generate_random_coords(AREA_SIZE, AREA_SIZE, 5)
    tag_coords = generate_inhomogeneous_ppp(regions)
    num_tags = len(tag_coords)
    unique_ids = generate_unique_ids(num_tags)

    tags = [{
        "id": uid, 
        "coords": (float(coord[0]), float(coord[1])), 
        "status": False
    } for uid, coord in zip(unique_ids, tag_coords)]
    return tags 

In [ ]:
def create_tags_homo(lam, area_size):
    """Tạo danh sách các tag với ID và tọa độ ngẫu nhiên trong vùng [0, area_size] theo Homogeneous Poisson Point Process"""
    tag_coords = generate_homogeneous_ppp(area_size, lam)
    num_tags = len(tag_coords)
    unique_ids = generate_unique_ids(num_tags)

    tags = [{
        "id": uid, 
        "coords": (float(coord[0]), float(coord[1])), 
        "status": False
    } for uid, coord in zip(unique_ids, tag_coords)]
    return tags

### SIC (Single Hash Information)

In [ ]:
used_numbers = set()


def generate_unique():
    """Sinh số ngẫu nhiên duy nhất"""
    while True:
        num = random.randint(0, 10**9)  
        if num not in used_numbers:
            used_numbers.add(num)
            return num

In [ ]:
def sic_hash(num_tags, r, id_tag, idx): 
    """Hash function cho SIC"""
    return (2 * id_tag + 3 * r + idx**2) % num_tags if num_tags > 0 else 0
    


def create_indicator_vector(tags, r):
    """Tạo indicator vector từ danh sách tag và tham số số ngẫu nhiên r."""
    num_tags = len(tags)
    indicator_vector = [0] * num_tags  # Khởi tạo vector với tất cả giá trị là 0
    slot_counts = [0] * num_tags       # Đếm số tag mỗi slot
    slot_owner = [False] * num_tags     # Trạng thái của tag trong mỗi slot (None nếu chưa có tag)

    # Đếm số tag trong mỗi slot
    for idx, tag in enumerate(tags):
        if not tag['status']:
            slot_time = sic_hash(num_tags, r, tag['id'], idx)
            slot_counts[slot_time] += 1
            slot_owner[slot_time] = tag

    # Xác định thành công/thất bại
    for slot, count in enumerate(slot_counts):
        if count == 1:  # duy nhất 1 tag -> thành công
            indicator_vector[slot] = 1
            slot_owner[slot]['status'] = True
        elif count > 1:  # nhiều hơn 1 tag -> va chạm
            indicator_vector[slot] = -1
            slot_owner[slot]['status'] = False
        else:
            indicator_vector[slot] = 0
            slot_owner[slot] = False

    return indicator_vector


In [ ]:
def split_indicator_vector(indicator_vector, bits_per_transmission=96):
    """Chia indicator vector thành các lần truyền có độ dài 96 bit bằng với kích thước ID của 1 tag."""
    transmissions = []
    for i in range(0, len(indicator_vector), bits_per_transmission):
        transmission = indicator_vector[i:i + bits_per_transmission]
        transmissions.append(transmission)
    return transmissions

### SIC (Successive Interference Cancellation)

### Thời gian và năng lượng

In [ ]:
# Với SIC (Single Hash Information)
 # Reader sẽ gửi indicator vector cho tag với tốc độ 26.5 kbps, chia indicator vector thành các lần truyền 96 bit để thời gian truyền là t_id 
 # Tag sẽ kiểm tra xem mình có được đọc thành công không rồi trả lời lại reader với bản tin 64 bit
 # Nếu không thành công, tag sẽ không trả lời

# VỚi PIC (Polling Information Collection)
 # Reader sẽ gửi ID 96 bit để gọi tên tag cụ thể mà reader đã phát hiện chưa thu thập được
 # Tag sẽ trả lời lại reader với bản tin 64 bit
 # Tag có thể phải nhận nhiều lần ID cho đến khi được đọc thành công

In [ ]:
def calculate_reader_transmission_time(transmissions, t_id, t_wait):
    """Tính thời gian reader truyền indicator vector đến tag."""
    total_time = t_id + t_wait  # Để frame đầu tiên để truyền số ngẫu nhiên r, loại thông tin cho mỗi round
    for tx in range(len(transmissions)):
        total_time += t_id + t_wait
        if tx == len(transmissions) - 1:
            total_time += t_id / 96 * (len(transmissions[tx]))   # Frame cuối cùng có thể không đủ 96 bit
    return total_time


def calculate_tag_transmission_time(indicator_vector, t_inf, t_wait):
    """Tính thời gian tag truyền bản tin cho reader."""
    total_time = 0
    for i in range(len(indicator_vector)):
        if indicator_vector[i] == 1:    # Khi slot thành công, truyền bản tin 64 bit
            total_time += t_inf
            if (i % 64 == 0) and (i != 0):
                total_time += t_wait 
    return total_time

In [ ]:
def calculate_lower_bound_time(num_tags, t_inf):
    """Tính thời gian lower bound."""
    return num_tags * t_inf

In [ ]:
def calculate_PIC_time(num_tags, t_inf, t_id):
    """Tính thời gian sử dụng giao thức PIC."""
    return (num_tags * t_inf) + (num_tags * t_id)

In [ ]:
def calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf):
    """Tính thời gian cho một vòng lặp SIC (Slotted ALOHA) bao gồm cả thời gian truyền và nhận của reader."""
    reader_time = calculate_reader_transmission_time(transmissions, t_id, t_wait)
    tag_time = calculate_tag_transmission_time(indicator_vector, t_inf, t_wait)

    return reader_time + tag_time

In [ ]:
def calculate_energy(power_level, num, time):
    """Tính năng lượng tiêu thụ"""
    return power_level * time * num

In [ ]:
def weight_power(P_high, P_low, n):
    """Tính trọng số công suất giữa hai mức công suất cao và thấp."""
    return (P_high / P_low) ** (2 / n)
    

def reader_density(weight, reader_radius, radius, radius_small, reader_coords):
    """Tính mật độ reader dựa trên:
    - weight: trọng số công suất giữa hai mức công suất cao và thấp
    - reader_radius: bán kính phủ sóng của reader
    - radius: bán kính mức công suất cao
    - radius_small: bán kính mức công suất thấp
    - reader_coords: tọa độ của các reader
    """
    densities = []
    for reader in range(len(reader_coords)):
        num_high = 0
        num_low = 0
        for other_reader in range(len(reader_coords)):
            if reader != other_reader:
                dist = math.dist(reader_coords[reader], reader_coords[other_reader])
                if dist <= reader_radius + radius_small:
                    num_low += 1
                elif dist <= reader_radius + radius:
                    num_high += 1
        density = (num_low + weight * num_high) / (weight * (len(reader_coords) - 1)) 
        densities.append(density)

    return densities

### Grouping

In [ ]:
def construct_activate_groups_system1(reader_coords, num_readers, reader_radius):
    """Tạo activation groups cho hệ thống 1:
    - Chia các reader thành các nhóm sao cho không có hai reader nào trong cùng một nhóm giao thoa với nhau.
    - Chỉ sử dụng 1 bán kính và công suất
    """
    unassigned_readers = list(range(num_readers))
    activation_groups = []
    
    while unassigned_readers:
        current_group = []
        seed_reader = unassigned_readers.pop(0)
        current_group.append((seed_reader, reader_coords[seed_reader], reader_radius))

        readers_to_check = unassigned_readers.copy()
        for r_idx in readers_to_check:
            if can_be_added(r_idx, current_group, reader_coords, reader_radius):
                current_group.append((r_idx, reader_coords[r_idx], reader_radius))
                unassigned_readers.remove(r_idx)

        activation_groups.append(current_group)
    
    return activation_groups

In [ ]:
def construct_activate_groups_system2(reader_coords, num_readers, reader_radius, reader_radius_small):
    """
    Tạo activation groups cho hệ thống 2:
    - Bắt đầu từ các nhóm của hệ thống 1
    - Mặc định các reader trong nhóm có bán kính lớn
    - Khi không thêm được reader nào nữa, giảm bán kính nhóm xuống nhỏ rồi thử lại
    """
    current_group = construct_activate_groups_system1(reader_coords, num_readers, reader_radius)
    activation_groups = []
    used_readers = set()

    for group in current_group:
        # Khởi tạo nhóm với bán kính lớn
        extended_group = [(r[0], reader_coords[r[0]], reader_radius) for r in group if r[0] not in used_readers]
        for r in extended_group:
            used_readers.add(r[0])

        added_any = False
        checked_readers = [r_idx for r_idx in range(num_readers) if r_idx not in used_readers]

        # --- BƯỚC 1: Thử thêm reader mới ---
        for r_idx in checked_readers[:]:
            # Ưu tiên thử bán kính lớn trước, nếu không được thì bán kính nhỏ
            if can_be_added(r_idx, extended_group, reader_coords, reader_radius):
                extended_group.append((r_idx, reader_coords[r_idx], reader_radius))
                used_readers.add(r_idx)
                added_any = True
            elif can_be_added(r_idx, extended_group, reader_coords, reader_radius_small):
                extended_group.append((r_idx, reader_coords[r_idx], reader_radius_small))
                used_readers.add(r_idx)
                added_any = True

        # --- BƯỚC 2: Nếu không thêm được ai, giảm bán kính toàn bộ nhóm ---
        if not added_any:
            extended_group = [(rid, coord, reader_radius_small) for rid, coord, _ in extended_group]

            # Thử lại với các reader chưa dùng
            checked_readers = [r for r in range(num_readers) if r not in used_readers]
            for r_idx in checked_readers[:]:
                if can_be_added(r_idx, extended_group, reader_coords, reader_radius):
                    extended_group.append((r_idx, reader_coords[r_idx], reader_radius))
                    used_readers.add(r_idx)
                elif can_be_added(r_idx, extended_group, reader_coords, reader_radius_small):
                    extended_group.append((r_idx, reader_coords[r_idx], reader_radius_small))
                    used_readers.add(r_idx)    

        activation_groups.append(extended_group)

    activation_groups = [group for group in activation_groups if group]    

    return activation_groups

In [ ]:
def construct_activate_groups_system3(reader_coords, num_readers, reader_radius, reader_radius_small):
    """Tạo activation groups cho hệ thống 3:
    - Bắt đầu từ reader chưa phân nhóm
    - Thử thêm reader mới với bán kính lớn trước, nếu không được thì bán kính nhỏ
    - Nếu không thêm được ai nữa, giảm bán kính toàn bộ nhóm rồi thử lại"""

    unassigned_readers = list(range(num_readers))
    activation_groups = []

    while unassigned_readers:
        current_group = []
        seed_reader = unassigned_readers.pop(0)
        current_group.append((seed_reader, reader_coords[seed_reader], reader_radius))
        added_any = False
        
        readers_to_check = unassigned_readers.copy()
        for r_idx in readers_to_check:                 
            # Thử thêm với bán kính lớn trước
            if can_be_added(r_idx, current_group, reader_coords, reader_radius):
                current_group.append((r_idx, reader_coords[r_idx], reader_radius))
                unassigned_readers.remove(r_idx)
                added_any = True
            # Nếu không được thì thử với bán kính nhỏ
            elif can_be_added(r_idx, current_group, reader_coords, reader_radius_small):
                current_group.append((r_idx, reader_coords[r_idx], reader_radius_small))
                unassigned_readers.remove(r_idx)
                added_any = True

            # Nếu không thêm được ai, giảm bán kính toàn bộ nhóm
            if not added_any:
                current_group = [(rid, coord, reader_radius_small) for rid, coord, _ in current_group]
                
                # Thử lại với các reader chưa dùng
                readers_to_check = unassigned_readers.copy()
                for r_idx in readers_to_check:
                    if can_be_added(r_idx, current_group, reader_coords, reader_radius):
                        current_group.append((r_idx, reader_coords[r_idx], reader_radius))
                        unassigned_readers.remove(r_idx)
                    elif can_be_added(r_idx, current_group, reader_coords, reader_radius_small):
                        current_group.append((r_idx, reader_coords[r_idx], reader_radius_small))
                        unassigned_readers.remove(r_idx)        

        activation_groups.append(current_group)

    return activation_groups

In [ ]:
def classificationPowerPhase(readers, tags, weight, reader_coords, reader_radius, reader_radius_small):
    """Phân loại công suất cho từng reader trong hệ thống 4 sử dụng FLC:
    - Tính mật độ reader xung quanh từng reader
    - Tính tỷ lệ thẻ gần và thẻ xa cho từng reader
    - Sử dụng FLC để phân loại công suất cho từng reader
    """
    power_decisions = []

    # Tính mật độ riêng cho từng reader 
    readerDensityLow = reader_density(weight, reader_radius_small, reader_radius, reader_radius_small, reader_coords)
    readerDensityHigh = reader_density(weight, reader_radius, reader_radius, reader_radius_small, reader_coords)

    for i, reader in enumerate(readers):
        numTags = len(confirm_tags_in_range(reader, tags, reader_coords, reader_radius))
        if numTags == 0:
            # Nếu reader không thấy thẻ nào => tắt reader
            power_decisions.append((reader, "Sleep"))
            continue

        tagCountNear = len(confirm_tags_in_range(reader, tags, reader_coords, reader_radius_small))
        tagCountFar = numTags - tagCountNear

        # Tính tỷ lệ
        near_ratio = tagCountNear / numTags
        far_ratio = tagCountFar / numTags

        # Tính điểm theo mật độ từng cấp
        score_low = classificationPower(near_ratio, far_ratio, readerDensityLow[i])
        score_high = classificationPower(near_ratio, far_ratio, readerDensityHigh[i])

        # So sánh và ra quyết định công suất
        power = "High Power" if score_high > score_low else "Low Power"
        power_decisions.append((reader, power))

    return power_decisions


def construct_activation_groups_system4(power_decisions, reader_coords, reader_radius, reader_radius_small):
    """Tạo activation groups cho hệ thống 4:
    - Bắt đầu từ reader chưa phân nhóm
    - Sử dụng bán kính tương ứng với quyết định công suất của reader (Sử dụng FLC để quyết định công suất)"""  
    reader_info = [
        (reader_id, reader_coords[i],
         reader_radius if power == "High Power" else reader_radius_small)
        for i, (reader_id, power) in enumerate(power_decisions)
        if power != "Sleep"
    ]

    if not reader_info:
        return []  # Trả về danh sách rỗng nếu không có reader nào hoạt động

    coords = [info[1] for info in reader_info]
    ungrouped = set(range(len(reader_info)))
    activation_groups = []

    while ungrouped:
        current = ungrouped.pop()
        seed_reader = reader_info[current]
        group = [seed_reader]
        to_remove = set()
        
        for other in list(ungrouped):
            r1 = reader_info[other][2]

            if can_be_added(other, group, coords, r1):
                group.append(reader_info[other])
                to_remove.add(other)


        ungrouped -= to_remove
        activation_groups.append(group)

    return activation_groups

In [ ]:
def group_coloring_4(power_decisions, reader_coords, reader_radius, reader_radius_small):
    """Phân nhóm các reader trong hệ thống 4 sử dụng thuật toán tô màu đồ thị DSATUR:
    - Tạo ma trận kề dựa trên giao thoa giữa các reader với bán kính tương ứng với quyết định công suất
    - Sử dụng thuật toán tô màu đồ thị DSATUR để phân nhóm các reader không giao thoa"""
    reader_info = [
        (reader_id, reader_coords[i],
         reader_radius if power == "High Power" else reader_radius_small)
        for i, (reader_id, power) in enumerate(power_decisions)
        if power != "Sleep"
    ]

    coords = [info[1] for info in reader_info]
    radius = [info[2] for info in reader_info]
    n = len(reader_info)
    
    # Xây dựng ma trận kề làm đầu vào cho thuật toán tô màu đồ thị
    matrix_adj = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(i + 1, n):
            r1 = reader_info[i][2]
            r2 = reader_info[j][2]
            if do_interfere(i, j, coords, (r1 + r2) / 2):
                matrix_adj[i][j] = 1
                matrix_adj[j][i] = 1
    # Xây dựng đồ thị
    G = nx.from_numpy_array(matrix_adj)

    # Tô màu đồ thị sử dụng thuật toán DSATUR
    coloring = nx.coloring.greedy_color(G, strategy="DSATUR")
    color_groups = {}

    for id, color in coloring.items():
        if color not in color_groups:
            color_groups[color] = []
        color_groups[color].append(reader_info[id])
       
    return list(color_groups.values())                              

In [ ]:
def create_grid(L, N=301):
    """Tạo lưới điểm trong phòng vuông kích thước LxL với N điểm mỗi chiều."""
    xs = np.linspace(0, L, N)
    ys = np.linspace(0, L, N)
    X, Y = np.meshgrid(xs, ys)
    return np.column_stack((X.ravel(), Y.ravel()))


# Tính bán kính nhỏ nhất cần thiết với 4 reader tại vị trí cho trước
def compute_required_radius(readers, grid):
    readers = np.array(readers)
    dists = np.sqrt(((grid[:, None, :] - readers[None, :, :]) ** 2).sum(axis=2))
    dmin = dists.min(axis=1)
    return dmin.max()   # điểm xa nhất → bán kính cần thiết


# Tìm d nhỏ nhất (dịch vào đường chéo)
def find_min_d_for_radius(r, L, grid):
    lo = 0.0
    hi = L / 2.0
    best_d = None

    for _ in range(35):   # số lần lặp cho độ chính xác ~1e-10
        mid = (lo + hi) / 2
        # 4 reader đặt đối xứng theo đường chéo
        readers = [
            (mid, mid),
            (L - mid, mid),
            (mid, L - mid),
            (L - mid, L - mid),
        ]
        rmin = compute_required_radius(readers, grid)

        if rmin <= r:
            best_d = mid
            hi = mid
        else:
            lo = mid

    return best_d

def result(r_min, r_max, r_step, L, grid):
    r_values = np.arange(r_min, r_max + 1e-9, r_step)

    results = []
    for r in r_values:
     d = find_min_d_for_radius(r, L, grid)
     results.append((f"{r:10.3f}", f"{d:10.3f}"))

    return results

def generate_reader_coords_symmetry(d):
    return np.array([[d, d], [21.2 -d, d], [d, 21.2 -d], [21.2 -d, 21.2 -d]])

## Thiết lập tham số

In [ ]:
# Thiết lập các tham số mô phỏng
# AREA_SIZE = 100
AREA_SIZE = 21.2
READER_RADIUS = 15.0
READER_RADIUS_SMALL = 10.0
NUM_TAGS = 5000

# Dựa trên đặc tả kỹ thuật Philips I-Code và tiêu chuẩn EPCGlobal Gen2
t_id = 96 / (26.5e3)             # Thời gian truyền dữ liệu từ reader đến tag
t_inf = 64 / (53e3)              # Thời gian truyền dữ liệu từ tag đến reader
t_wait = 302e-6                  # Thời gian chờ giữa các lần truyền

Preader_tx1 = 0.6                # Công suất reader truyền dữ liệu cho vùng bán kính lớn
Preader_tx2 = 0.01122            # Công suất reader truyền dữ liệu cho vùng bán kính nhỏ
Preader_rx = 0.494               # Công suất reader nhận dữ liệu
Preader_wait = 0.114             # Công suất reader chờ

Ptag_tx = 0.035                  # Công suất tag truyền dữ liệu
Ptag_rx = 0.028                  # Công suất tag nhận dữ liệu

threshold = list(np.linspace(0.05, 1, 100, endpoint=True))  # Ngưỡng để chuyển sang PIC

# Tọa độ cho các reader
# reader_coords = generate_random_reader_coords(10, AREA_SIZE)
reader_coords = np.array([[0, 0],
       [21.2, 0],
       [0, 21.2],
       [ 21.2, 21.2]])
# print(reader_coords)

weight = weight_power(Preader_tx1, Preader_tx2, 4)



In [ ]:
# Tạo regions
regions = create_regions(AREA_SIZE, 15, (0.5, 1), seed=42)
print("Regions:", regions)

In [ ]:
# # Tạo danh sách các tag với ID và tọa độ ngẫu nhiên trong vùng reader phủ sóng
# tags = create_tags(NUM_TAGS, reader_coords, READER_RADIUS)

# # Tách tọa độ tag thành hai danh sách riêng biệt
# tag_x = [tag['coords'][0] for tag in tags]
# tag_y = [tag['coords'][1] for tag in tags]

# tag_coords = list(zip(tag_x, tag_y))

In [ ]:
# Tạo danh sách các tag với ID và tọa độ ngẫu nhiên trong vùng [0, area_size]
tags = create_tags_inhomo(regions)

tag_x = [tag['coords'][0] for tag in tags]
tag_y = [tag['coords'][1] for tag in tags]
tag_coords = list(zip(tag_x, tag_y))

In [ ]:
# # Tạo bản đồ công suất ban đầu
# power_decisions = classificationPowerPhase(list(range(len(reader_coords))), tags, weight, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
# print("Power decisions:", power_decisions)

In [ ]:
# activation_groups1 = construct_activate_groups_system1(reader_coords, len(reader_coords), READER_RADIUS)
# only_ids1 = [[reader[0] for reader in group] for group in activation_groups1]
# print(f"Các nhóm kích hoạt đã được tạo: {only_ids1}")

# activation_groups2 = construct_activate_groups_system2(reader_coords, len(reader_coords), READER_RADIUS, READER_RADIUS_SMALL)
# only_ids2 = [[reader[0] for reader in group] for group in activation_groups2]
# print(f"Các nhóm kích hoạt đã được tạo: {only_ids2}")

# activation_groups3 = construct_activate_groups_system3(reader_coords, len(reader_coords), READER_RADIUS, READER_RADIUS_SMALL)
# only_ids3 = [[reader[0] for reader in group] for group in activation_groups3]
# print(f"Các nhóm kích hoạt đã được tạo: {only_ids3}")

# activation_groups4 = construct_activation_groups_system4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
# only_ids4 = [[reader[0] for reader in group] for group in activation_groups4]
# print(f"Các nhóm kích hoạt đã được tạo: {only_ids4}")

# activation_group_coloring_4 = group_coloring_4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
# only_ids_coloring4 = [[reader[0] for reader in group] for group in group_coloring_4]
# print(f"Các nhóm kích hoạt đã được tạo bằng tô màu đồ thị: {only_ids_coloring4}")

## Vẽ đồ thị phân nhóm


### Cho 4 hệ thống sử dụng cách phân nhóm tuần tự

In [ ]:
# # Vẽ đồ thị
# fig, axs = plt.subplots(2, 2, figsize=(16, 8))
# ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]

# # Định nghĩa màu cho các nhóm
# colors = ['blue', 'green', 'orange', 'purple', 'brown', 'cyan', 'magenta', 'yellow', 'lime', 'pink']

# # Vẽ các vùng phủ sóng với màu tương ứng của nhóm
# for i, group in enumerate(activation_groups1):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         circle1 = Circle((x, y), READER_RADIUS, facecolor='none', edgecolor=color, linewidth=1.5)
#         ax1.add_patch(circle1)

# for i, group in enumerate(activation_groups2):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         r_current = reader_info[2]

#         circle1 = Circle((x, y), r_current, facecolor='none', edgecolor=color, linewidth=1.5)  
#         ax2.add_patch(circle1)

#         if r_current == READER_RADIUS:
#             r_other = READER_RADIUS_SMALL
#         else:
#             r_other = READER_RADIUS

#         circle2 = Circle((x, y), r_other, facecolor='none', edgecolor='black', linestyle='--', alpha=0.3)    
#         ax2.add_patch(circle2)

# for i, group in enumerate(activation_groups3):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         circle1 = Circle((x, y), reader_info[2], facecolor='none', edgecolor=color, linewidth=1.5) 
#         ax3.add_patch(circle1)

#         if reader_info[2] == READER_RADIUS:
#             r_other = READER_RADIUS_SMALL
#         else:
#             r_other = READER_RADIUS

#         circle2 = Circle((x, y), r_other, facecolor='none', edgecolor='black', linestyle='--', alpha=0.3)    
#         ax3.add_patch(circle2)

# for i, group in enumerate(activation_groups4):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         r_current = reader_info[2]
#         circle1 = Circle((x, y), r_current, facecolor='none', edgecolor=color, linewidth=1.5)
#         ax4.add_patch(circle1)

#         if r_current == READER_RADIUS:
#             r_other = READER_RADIUS_SMALL
#         else:
#             r_other = READER_RADIUS

#         circle2 = Circle((x, y), r_other, facecolor='none', edgecolor='black', linestyle='--', alpha=0.3)
#         ax4.add_patch(circle2)

# # Vẽ các tag và reader
# ax1.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax1.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')

# ax2.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax2.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')

# ax3.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax3.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')

# ax4.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax4.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')

# # Hoàn thiện đồ thị
# for ax in [ax1, ax2, ax3, ax4]:
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlim(0, AREA_SIZE)
#     ax.set_ylim(0, AREA_SIZE)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.legend()
#     ax.set_title('Activation Groups Visualization System 1' if ax is ax1 else 'Activation Groups Visualization System 2' if ax is ax2 
#                  else 'Activation Groups Visualization System 3' if ax is ax3 else 'Activation Groups Visualization System 4')

# plt.show()


### So sánh hệ thống 4 trong 2 trường hợp phân nhóm tuần tư và tô màu đồ thị

In [ ]:
# fig, axs = plt.subplots(1, 2,figsize=(16, 8))
# ax1, ax2 = axs[0], axs[1]

# for i, group in enumerate(activation_groups4):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         r_current = reader_info[2]
#         circle1 = Circle((x, y), r_current, facecolor='none', edgecolor=color, linewidth=1.5)
#         ax1.add_patch(circle1)

#         if r_current == READER_RADIUS:
#             r_other = READER_RADIUS_SMALL
#         else:
#             r_other = READER_RADIUS

#         circle2 = Circle((x, y), r_other, facecolor='none', edgecolor='black', linestyle='--', alpha=0.3)
#         ax1.add_patch(circle2)

# # Vẽ các vùng phủ sóng với màu tương ứng của nhóm
# for i, group in enumerate(activation_group_coloring_4):
#     color = colors[i % len(colors)]
#     for reader_info in group:
#         x, y = reader_info[1]
#         r_current = reader_info[2]
#         circle1 = Circle((x, y), r_current, facecolor='none', edgecolor=color, linewidth=1.5)
#         ax2.add_patch(circle1)

#         if r_current == READER_RADIUS:
#             r_other = READER_RADIUS_SMALL
#         else:
#             r_other = READER_RADIUS

#         circle2 = Circle((x, y), r_other, facecolor='none', edgecolor='black', linestyle='--', alpha=0.3)
#         ax2.add_patch(circle2)

# # ax1.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax1.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')
# # ax2.scatter(tag_x, tag_y, s=1, color='lightgray', label='Tag') # Cho tag màu xám để nổi bật các vùng
# ax2.scatter(reader_coords[:, 0], reader_coords[:, 1], s=50, color='red', marker='.', label='Reader')

# for ax in [ax1, ax2]:
#     ax.set_aspect('equal', adjustable='box')
#     ax.set_xlim(0, AREA_SIZE)
#     ax.set_ylim(0, AREA_SIZE)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.legend()
#     ax.set_title('Activation Groups Visualization System 4 - Graph Coloring' if ax is ax2 else 'Activation Groups Visualization System 4')        

In [ ]:
# num_groups1 = []
# num_groups2 = []
# for n in range(6, 20):
#     reader_coords = generate_random_reader_coords(n, AREA_SIZE)
#     power_decisions = classificationPowerPhase(list(range(len(reader_coords))), tags, weight, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)

#     activation_groups4 = construct_activation_groups_system4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
#     only_ids4 = [[reader[0] for reader in group] for group in activation_groups4]
    
#     groups_coloring_4 = group_coloring_4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
#     only_ids_coloring4 = [[reader[0] for reader in group] for group in groups_coloring_4]

#     num_groups1.append(len(only_ids4))
#     num_groups2.append(len(only_ids_coloring4))

# plt.figure(figsize=(10, 6))
# plt.plot(range(6, 20), num_groups1, marker='o', label='System 4 Activation Groups')
# plt.plot(range(6, 20), num_groups2, marker='s', label='System 4 Graph Coloring Groups')
# plt.xlabel('Number of Readers')    
# plt.ylabel('Number of Groups')
# plt.title('Comparison of Activation Groups and Graph Coloring Groups')
# plt.legend()
# plt.grid(True)
# plt.show()

## Tính thời gian và năng lượng cho từng hệ thống

### Các tag nằm hoàn toàn trong vùng phủ sóng của reader

In [ ]:
# # Thời gian truyền của lower bound
# lower_bound_time = 0
# for reader in range(len(reader_coords)):
#     num_tags = len(confirm_tags_in_range(reader, tags, reader_coords, READER_RADIUS))
#     lower_bound_time = max(lower_bound_time, calculate_lower_bound_time(num_tags, t_inf))
# print(f"Thời gian truyền lower bound: {lower_bound_time} (s)")


In [ ]:
# # Thời gian truyền của system 1
# time1 = []
# energy_reader1 = []
# average_energy_tag1 = []
# for th in threshold:
#  system1_time = 0
#  total_energy_reader1 = 0
#  total_energy_tag_sic1 = 0
#  total_energy_tag_pic1 = 0
#  for group in sorted(activation_groups1, key=len, reverse=True):
#      group_time1 = 0
#      group_energy_reader1 = 0
#      group_energy_tag_sic1 = 0
#      group_energy_tag_pic1 = 0
#      tx_energy_reader1 = 0
#      rx_energy_reader1 = 0
#      wait_energy_reader1 = 0

#      tx_energy_tag_sic1 = 0
#      rx_energy_tag_sic1 = 0
#      tx_energy_tag_pic1 = 0
#      rx_energy_tag_pic1 = 0
    
#      # Xác định tags trong vùng group
#      group_tags1 = confirm_tags_in_group_range(group, tags, reader_coords, READER_RADIUS)
#      ungathered_tags1 = {reader: get_unread_tags_in_range(reader, tags, reader_coords, READER_RADIUS) 
#                         for reader in group}

#      readers_sic_time1 = [0] * len(group)
#      # Xử lý SIC cho tất cả reader song song
#      while any((len(ungathered_tags1[r]) / len(group_tags1) >= th) for r in group):
#          for idx, reader in enumerate(group):
#              rd = generate_unique()
#              reader_tags = confirm_tags_in_range(reader, tags, reader_coords, READER_RADIUS)
#              indicator_vector = create_indicator_vector(reader_tags, rd)
#              transmissions = split_indicator_vector(indicator_vector)

#              tx_energy_reader1 += Preader_tx2 * len(indicator_vector) * (t_id + t_wait)
#              rx_energy_reader1 += Preader_rx * sum(1 for x in indicator_vector if x == 1) * (t_inf + t_wait)
#              wait_energy_reader1 += Preader_wait * (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)) * t_inf

#              tx_energy_tag_sic1 += Ptag_tx * sum(1 for x in indicator_vector if x == 1) * t_inf
#              rx_energy_tag_sic1 += Ptag_rx * len(indicator_vector) * t_id

#              if (reader_tags):
#                  reader_time = calculate_reader_round_time(
#                      transmissions, t_id, t_wait, indicator_vector, t_inf
#                  )
#                  readers_sic_time1[idx] += reader_time
#              else:
#                  readers_sic_time1[idx] += 0
#              # cập nhật tag chưa đọc của reader
#              ungathered_tags1[reader] = get_unread_tags_in_range(reader, tags, reader_coords, READER_RADIUS)

#          # Group phải chờ reader lâu nhất trong         
#         #  group_time1 = max(readers_sic_time1) if readers_sic_time1 else 0

#      # Xử lý PIC cho tất cả reader song song
#      readers_pic_time1 = [0] * len(group)
#      for idx, reader in enumerate(group):
#         if ungathered_tags1[reader]:  # nếu vẫn còn tag chưa đọc
#             pic_time = calculate_PIC_time(len(ungathered_tags1[reader]), t_inf, t_id)
            
#             tx_energy_reader1 += Preader_tx1 * len(ungathered_tags1[reader]) * (t_id + t_wait)
#             rx_energy_reader1 += Preader_rx * len(ungathered_tags1[reader]) * (t_inf + t_wait)

#             tx_energy_tag_pic1 += Ptag_tx * len(ungathered_tags1[reader]) * t_inf
#             rx_energy_tag_pic1 += Ptag_rx * len(ungathered_tags1[reader]) / 2 * (t_id + t_inf)
#             for tag in ungathered_tags1[reader]:
#                 tag['status'] = True
#             readers_pic_time1[idx] = pic_time
#         else:
#             readers_pic_time1[idx] = 0  # Nếu không còn tag, thời gian PIC là 0

#      readers_time_1 = [0] * len(group)
#      for idx, reader in enumerate(group):
#          readers_time_1[idx] = readers_sic_time1[idx] + readers_pic_time1[idx]

#      group_time1 = max(readers_time_1) if readers_time_1 else 0
#      group_energy_reader1 += (tx_energy_reader1 + rx_energy_reader1 + wait_energy_reader1)
#      group_energy_tag_sic1 += (tx_energy_tag_sic1 + rx_energy_tag_sic1)
#      group_energy_tag_pic1 += (tx_energy_tag_pic1 + rx_energy_tag_pic1)

#      # Cộng thời gian group này vào hệ thống
#      system1_time += group_time1
#      total_energy_reader1 += group_energy_reader1
#      total_energy_tag_sic1 += group_energy_tag_sic1
#      total_energy_tag_pic1 += group_energy_tag_pic1

#  for tag in tags:  
#     tag['status'] = False

#  time1.append(system1_time)
#  energy_reader1.append(total_energy_reader1)
#  average_energy_tag1.append(1 / len(tags) * total_energy_tag_sic1 + total_energy_tag_pic1)

# #  print(f"Thời gian truyền dữ liệu của hệ thống 1: {system1_time} (s)")

In [ ]:
# # Thời gian truyền của system 2
# time2 = []
# energy_reader2 = []
# average_energy_tag2 = []
# for th in threshold:
#  system2_time = 0
#  total_energy_reader2 = 0
#  total_energy_tag_sic2 = 0
#  total_energy_tag_pic2 = 0
#  for group in sorted(activation_groups2, key=len, reverse=True):
#      group_time2 = 0
#      group_energy_reader2 = 0
#      group_energy_tag_sic2 = 0
#      group_energy_tag_pic2 = 0
#      tx_energy_reader2 = 0
#      rx_energy_reader2 = 0
#      wait_energy_reader2 = 0

#      tx_energy_tag_sic2 = 0
#      rx_energy_tag_sic2 = 0
#      tx_energy_tag_pic2 = 0
#      rx_energy_tag_pic2 = 0
     

#      # Xác định tags trong vùng group
#      group_tags2 = confirm_tags_in_group_range(group, tags, reader_coords, READER_RADIUS_SMALL)
#      ungathered_tags_small = {reader: get_unread_tags_in_range(reader, tags, reader_coords, READER_RADIUS_SMALL)
#                        for reader in group}
#      readers_sic_time2 = [0] * len(group)
#      # Vòng lặp round cho group
#      while any((len(ungathered_tags_small[r]) / len(group_tags2) >= th) for r in group):
#          for idx, reader in enumerate(group):
#              if ungathered_tags_small[reader]:
#                  rd = generate_unique()
#                  reader_tags = confirm_tags_in_range(reader, tags, reader_coords, READER_RADIUS_SMALL)
#                  indicator_vector = create_indicator_vector(reader_tags, rd)
#                  transmissions = split_indicator_vector(indicator_vector)

#                  tx_energy_reader2 += Preader_tx2 * len(indicator_vector) * (t_id + t_wait)
#                  rx_energy_reader2 += Preader_rx * sum(1 for x in indicator_vector if x == 1) * (t_inf + t_wait)
#                  wait_energy_reader2 += Preader_wait * (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)) * t_inf

#                  rx_energy_tag_sic2 += Ptag_tx * len(indicator_vector) * t_id
#                  tx_energy_tag_sic2 += Ptag_rx * sum(1 for x in indicator_vector if x == 1) * t_inf


#                  if (reader_tags):
#                      reader_time = calculate_reader_round_time(
#                          transmissions, t_id, t_wait, indicator_vector, t_inf
#                      )
#                      readers_sic_time2[idx] += reader_time
#                  else:
#                      readers_sic_time2[idx] += 0

#              # cập nhật tag chưa đọc của reader
#              ungathered_tags_small[reader] = get_unread_tags_in_range(reader, tags, reader_coords, READER_RADIUS_SMALL)

#          # Group phải chờ reader lâu nhất
#         #  group_time2 = max(readers_time2) if readers_time2 else 0

#      # Xử lý PIC cho tất cả reader song song
#      readers_pic_time_small = [0] * len(group)
#      for idx, reader in enumerate(group):
#         if ungathered_tags_small[reader]:  # nếu vẫn còn tag chưa đọc
#             pic_time = calculate_PIC_time(len(ungathered_tags_small[reader]), t_inf, t_id)

#             tx_energy_reader2 += Preader_tx1 * len(ungathered_tags_small[reader]) * (t_id + t_wait)
#             rx_energy_reader2 += Preader_rx * len(ungathered_tags_small[reader]) * (t_inf + t_wait)

#             tx_energy_tag_pic2 += Ptag_tx * len(ungathered_tags_small[reader]) * t_inf
#             rx_energy_tag_pic2 += Ptag_rx * len(ungathered_tags_small[reader]) / 2 * (t_id + t_inf)

#             for tag in ungathered_tags_small[reader]:
#                 tag['status'] = True
#             readers_pic_time_small[idx] = pic_time
#         else:
#             readers_pic_time_small[idx] = 0  # Nếu không còn tag, thời gian PIC là 0

#     #  group_time2 += max(readers_pic_time_small) if readers_pic_time_small else 0

#      readers_pic_time2 = [0] * len(group)
#      for idx, reader in enumerate(group):
#         ungathered_tags = get_unread_tags_in_range(reader, tags, reader_coords, READER_RADIUS)
#         pic_time = calculate_PIC_time(len(ungathered_tags), t_inf, t_id)

#         tx_energy_reader2 += Preader_tx1 * len(ungathered_tags) * (t_id + t_wait)
#         rx_energy_reader2 += Preader_rx * len(ungathered_tags) * (t_inf + t_wait)

#         tx_energy_tag_pic2 += Ptag_tx * len(ungathered_tags) * t_inf
#         rx_energy_tag_pic2 += Ptag_rx * len(ungathered_tags) / 2 * (t_id + t_inf)

#         for tag in ungathered_tags:
#             tag['status'] = True
#         readers_pic_time2[idx] = pic_time

#      readers_time_2 = [0] * len(group)
#      for idx, reader in enumerate(group):
#          readers_time_2[idx] = readers_sic_time2[idx] + readers_pic_time2[idx] + readers_pic_time_small[idx]

#      group_time2 = max(readers_time_2) if readers_time_2 else 0
#      group_energy_reader2 += (tx_energy_reader2 + rx_energy_reader2 + wait_energy_reader2)
#      group_energy_tag_sic2 += (tx_energy_tag_sic2 + rx_energy_tag_sic2)
#      group_energy_tag_pic2 += (tx_energy_tag_pic2 + rx_energy_tag_pic2)

#     # Cộng thời gian group này vào hệ thống
#      system2_time += group_time2
#      total_energy_reader2 += group_energy_reader2
#      total_energy_tag_sic2 += group_energy_tag_sic2
#      total_energy_tag_pic2 += group_energy_tag_pic2

#  for tag in tags:  
#     tag['status'] = False    
    
#  time2.append(system2_time) 
#  energy_reader2.append(total_energy_reader2)
#  average_energy_tag2.append(1 / len(tags) * total_energy_tag_sic2 + total_energy_tag_pic2)
# # print(f"Thời gian truyền dữ liệu của hệ thống 2: {system2_time} (s)")

### Các tag nằm ngẫu nhiên, số lượng tag theo Poisson

In [ ]:
# lower_bound_time = 0
# for reader in range(len(reader_coords)):
#     num_tags = len(confirm_tags_in_range(reader, tags, reader_coords, READER_RADIUS))
#     lower_bound_time = max(lower_bound_time, calculate_lower_bound_time(num_tags, t_inf))
# print(f"Thời gian truyền lower bound: {lower_bound_time} (s)")    

#### Thời gian và năng lượng trên các threshold

In [ ]:
def system1(activation_groups1, tags, reader_coords, reader_radius):
 """Thời gian truyền và năng lượng tiêu thụ của reader và tag trong system 1"""
 time1 = []
 energy_reader1 = []
 average_energy_tag1 = []

 a_system1_time = 0
 a_energy_reader1 = 0
 a_energy_tag1 = 0

 for th in threshold:
  system1_time = 0
  total_energy_reader1 = 0
  total_energy_tag_sic1 = 0
  total_energy_tag_pic1 = 0
  for group in sorted(activation_groups1, key=len, reverse=True):
      group_time1 = 0
      readers_time_1 = [0] * len(reader_coords)
      readers_sic_time1 = [0] * len(reader_coords)
      readers_pic_time1 = [0] * len(reader_coords)

      group_energy_reader1 = 0
      group_energy_tag_sic1 = 0
      group_energy_tag_pic1 = 0
      tx_energy_reader1 = 0
      rx_energy_reader1 = 0
      wait_energy_reader1 = 0

      tx_energy_tag_sic1 = 0
      rx_energy_tag_sic1 = 0
      tx_energy_tag_pic1 = 0
      rx_energy_tag_pic1 = 0
    
      # Xác định tags trong vùng group
      group_tags1 = confirm_tags_in_group_range(group, tags)
      if not group_tags1:
          continue
      reader_tags1 = {reader[0]: confirm_tags_in_range(reader[0], tags, reader_coords, reader_radius) 
                        for reader in group}

      unread_tags = copy.deepcopy(reader_tags1)
      # Xử lý SIC cho tất cả reader song song
      while any((len(unread_tags[r[0]]) / len(group_tags1) > th) for r in group):
          for reader in group:
              unread_tags[reader[0]] = [t for t in reader_tags1[reader[0]] if not t['status']]
              if not unread_tags[reader[0]]:
                  readers_time_1[reader[0]] = 0              
                  continue
             
              rd = generate_unique()
              indicator_vector = create_indicator_vector(unread_tags[reader[0]], rd)
              transmissions = split_indicator_vector(indicator_vector)

              tx_energy_reader1 += calculate_energy(Preader_tx1, len(indicator_vector), (t_id + t_wait))
              rx_energy_reader1 += calculate_energy(Preader_rx, sum(1 for x in indicator_vector if x == 1), (t_inf + t_wait))
              wait_energy_reader1 += calculate_energy(Preader_wait, (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)), t_inf)

              tx_energy_tag_sic1 += calculate_energy(Ptag_tx, sum(1 for x in indicator_vector if x == 1), t_inf)
              rx_energy_tag_sic1 += calculate_energy(Ptag_rx, len(indicator_vector), t_id)
             
              reader_time = calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf)
              readers_sic_time1[reader[0]] += reader_time
             
              # cập nhật tag chưa đọc của reader
              unread_tags[reader[0]] = [t for t in unread_tags[reader[0]] if not t['status']]


      # Xử lý PIC cho tất cả reader song song    
      for reader in group:
         if unread_tags[reader[0]]:  # nếu vẫn còn tag chưa đọc
             pic_time = calculate_PIC_time(len(unread_tags[reader[0]]), t_inf, t_id)

             tx_energy_reader1 += calculate_energy(Preader_tx1, len(unread_tags[reader[0]]), (t_id + t_wait))
             rx_energy_reader1 += calculate_energy(Preader_rx, len(unread_tags[reader[0]]), (t_inf + t_wait))

             tx_energy_tag_pic1 += calculate_energy(Ptag_tx, len(unread_tags[reader[0]]), t_inf)
             rx_energy_tag_pic1 += calculate_energy(Ptag_rx, len(unread_tags[reader[0]]) / 2, (t_id + t_inf))
             for tag in unread_tags[reader[0]]:
                 tag['status'] = True
             readers_pic_time1[reader[0]] = pic_time
         else:
             readers_pic_time1[reader[0]] = 0  # Nếu không còn tag, thời gian PIC là 0

     
      for reader in group:
          readers_time_1[reader[0]] = readers_sic_time1[reader[0]] + readers_pic_time1[reader[0]]

      group_time1 = max(readers_time_1) if readers_time_1 else 0
      group_energy_reader1 += (tx_energy_reader1 + rx_energy_reader1 + wait_energy_reader1)
      group_energy_tag_sic1 += (tx_energy_tag_sic1 + rx_energy_tag_sic1)
      group_energy_tag_pic1 += (tx_energy_tag_pic1 + rx_energy_tag_pic1)

      # Cộng thời gian group này vào hệ thống
      system1_time += group_time1
      total_energy_reader1 += group_energy_reader1
      total_energy_tag_sic1 += group_energy_tag_sic1
      total_energy_tag_pic1 += group_energy_tag_pic1

  for tag in tags:  
     tag['status'] = False
     rd = None
     

  time1.append(system1_time)
  energy_reader1.append(total_energy_reader1)
  average_energy_tag1.append((total_energy_tag_sic1 + total_energy_tag_pic1) / len(tags))

 return time1, energy_reader1, average_energy_tag1

In [ ]:
def system2(activation_groups2, tags, reader_coords, reader_radius, reader_radius_small):
 # Thời gian truyền của system 2
 time2 = []
 energy_reader2 = []
 average_energy_tag2 = []

 a_system2_time = 0
 a_energy_reader2 = 0
 a_energy_tag2 = 0

 for th in threshold:
  system2_time = 0
  total_energy_reader2 = 0
  total_energy_tag_sic2 = 0
  total_energy_tag_pic2 = 0
  for group in sorted(activation_groups2, key=len, reverse=True):
      group_time2 = 0
      readers_time_2 = [0] * len(reader_coords)
      readers_pic_time2 = [0] * len(reader_coords)
      readers_pic_time_current2 = [0] * len(reader_coords)
      readers_sic_time2 = [0] * len(reader_coords)

      group_energy_reader2 = 0
      group_energy_tag_sic2 = 0
      group_energy_tag_pic2 = 0
      tx_energy_reader2 = 0
      rx_energy_reader2 = 0
      wait_energy_reader2 = 0

      tx_energy_tag_sic2 = 0
      rx_energy_tag_sic2 = 0
      tx_energy_tag_pic2 = 0
      rx_energy_tag_pic2 = 0
     

      # Xác định tags trong vùng group
      group_tags2 = confirm_tags_in_group_range(group, tags)
      if not group_tags2:
         continue
     
      reader_tags_current = {reader_info[0]: confirm_tags_in_range(reader_info[0], tags, reader_coords, reader_info[2])
                       for reader_info in group}
      unread_tags_current = copy.deepcopy(reader_tags_current)  
      # Vòng lặp round cho group
      while any((len(unread_tags_current[r[0]]) / len(group_tags2) > th) for r in group):
          for reader_info in group:

            unread_tags_current[reader_info[0]] = [t for t in reader_tags_current[reader_info[0]] if not t['status']]
            if not unread_tags_current[reader_info[0]]:
                continue

            rd = generate_unique()
            indicator_vector = create_indicator_vector(unread_tags_current[reader_info[0]], rd)
            transmissions = split_indicator_vector(indicator_vector)
            power = Preader_tx1 if reader_info[2] == reader_radius else Preader_tx2
 
            tx_energy_reader2 += calculate_energy(power, len(indicator_vector), (t_id + t_wait))
            rx_energy_reader2 += calculate_energy(Preader_rx, sum(1 for x in indicator_vector if x == 1), (t_inf + t_wait))
            wait_energy_reader2 += calculate_energy(Preader_wait, (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)), t_inf)

            rx_energy_tag_sic2 += calculate_energy(Ptag_rx, len(indicator_vector), t_id)
            tx_energy_tag_sic2 += calculate_energy(Ptag_tx, sum(1 for x in indicator_vector if x == 1), t_inf)
       
            reader_time = calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf)
            readers_sic_time2[reader_info[0]] += reader_time
            

                  # cập nhật tag chưa đọc của reader
            unread_tags_current[reader_info[0]] = [t for t in reader_tags_current[reader_info[0]] if not t['status']]


      # Xử lý PIC cho tất cả reader song song     
      for reader_info in group:
         if unread_tags_current[reader_info[0]]:  # nếu vẫn còn tag chưa đọc
             pic_time = calculate_PIC_time(len(unread_tags_current[reader_info[0]]), t_inf, t_id)
             power = Preader_tx1 if reader_info[2] == reader_radius else Preader_tx2

             tx_energy_reader2 += calculate_energy(power, len(unread_tags_current[reader_info[0]]), (t_id + t_wait))
             rx_energy_reader2 += calculate_energy(Preader_rx, len(unread_tags_current[reader_info[0]]), (t_inf + t_wait))

             tx_energy_tag_pic2 += calculate_energy(Ptag_tx, len(unread_tags_current[reader_info[0]]), t_inf)
             rx_energy_tag_pic2 += calculate_energy(Ptag_rx, len(unread_tags_current[reader_info[0]]) / 2, (t_id + t_inf))

             for tag in unread_tags_current[reader_info[0]]:
                 tag['status'] = True
                
             readers_pic_time_current2[reader_info[0]] = pic_time
         else:
             readers_pic_time_current2[reader_info[0]] = 0  # Nếu không còn tag, thời gian PIC là 0

      for reader_info in group:
          readers_time_2[reader_info[0]] = readers_sic_time2[reader_info[0]] + readers_pic_time_current2[reader_info[0]]       

      for reader_info in group:
         radius_other = reader_radius if reader_info[2] == reader_radius_small else reader_radius_small
         power = Preader_tx1 if radius_other == reader_radius else Preader_tx2
         reader_tag = confirm_tags_in_range(reader_info[0], tags, reader_coords, radius_other)
         ungathered_tags = [t for t in reader_tag if not t['status']]
         pic_time = calculate_PIC_time(len(ungathered_tags), t_inf, t_id)

         tx_energy_reader2 += calculate_energy(power, len(ungathered_tags), (t_id + t_wait))
         rx_energy_reader2 += calculate_energy(Preader_rx, len(ungathered_tags), (t_inf + t_wait))

         tx_energy_tag_pic2 += calculate_energy(Ptag_tx, len(ungathered_tags), t_inf)
         rx_energy_tag_pic2 += calculate_energy(Ptag_rx, len(ungathered_tags) / 2, (t_id + t_inf))

         for tag in ungathered_tags:
             tag['status'] = True
         readers_pic_time2[reader_info[0]] = pic_time

     
      group_time2 = max(readers_time_2) if readers_time_2 else 0
      for reader_info in group:
          group_time2 += readers_pic_time2[reader_info[0]]
         
      group_energy_reader2 += (tx_energy_reader2 + rx_energy_reader2 + wait_energy_reader2)
      group_energy_tag_sic2 += (tx_energy_tag_sic2 + rx_energy_tag_sic2)
      group_energy_tag_pic2 += (tx_energy_tag_pic2 + rx_energy_tag_pic2)

     # Cộng thời gian group này vào hệ thống
      system2_time += group_time2
      total_energy_reader2 += group_energy_reader2
      total_energy_tag_sic2 += group_energy_tag_sic2
      total_energy_tag_pic2 += group_energy_tag_pic2

  for tag in tags:  
     tag['status'] = False    
     rd = None
    
  time2.append(system2_time) 
  energy_reader2.append(total_energy_reader2)
  average_energy_tag2.append(1 / len(tags) * (total_energy_tag_sic2 + total_energy_tag_pic2))

 
 return time2, energy_reader2, average_energy_tag2


In [ ]:
def system3(activation_groups3, tags, reader_coords, reader_radius, reader_radius_small):
    # Thời gian truyền của system 3
 time3 = []
 energy_reader3 = []
 average_energy_tag3 = []

 a_system3_time = 0
 a_energy_reader3 = 0
 a_energy_tag3 = 0

 for th in threshold:
  system3_time = 0
  total_energy_reader3 = 0
  total_energy_tag_sic3 = 0
  total_energy_tag_pic3 = 0
  for group in sorted(activation_groups3, key=len, reverse=True):
      group_time3 = 0
      readers_time_3 = [0] * len(reader_coords)
      readers_sic_time3 = [0] * len(reader_coords)
      readers_pic_time_current_3 = [0] * len(reader_coords)
      readers_pic_time3 = [0] * len(reader_coords)

      group_energy_reader3 = 0
      group_energy_tag_sic3 = 0
      group_energy_tag_pic3 = 0
      tx_energy_reader3 = 0
      rx_energy_reader3 = 0
      wait_energy_reader3 = 0

      tx_energy_tag_sic3 = 0
      rx_energy_tag_sic3 = 0
      tx_energy_tag_pic3 = 0
      rx_energy_tag_pic3 = 0
     

     # Xác định tags trong vùng group
      group_tags3 = confirm_tags_in_group_range(group, tags)
      if not group_tags3:
        continue

      reader_tags_current = {reader_info[0]: confirm_tags_in_range(reader_info[0],tags, reader_coords, reader_info[2])
                       for reader_info in group}
      unread_tags_current = copy.deepcopy(reader_tags_current)

     # Vòng lặp round cho group
      while any((len(unread_tags_current[r[0]]) / len(group_tags3) > th) for r in group):
          for reader_info in group:

                  unread_tags_current[reader_info[0]] = [t for t in reader_tags_current[reader_info[0]] if not t['status']]
                  if not unread_tags_current[reader_info[0]]:
                      continue
                 
                  rd = generate_unique()
                  indicator_vector = create_indicator_vector(unread_tags_current[reader_info[0]], rd)
                  transmissions = split_indicator_vector(indicator_vector)
                  power = Preader_tx1 if reader_info[2] == reader_radius else Preader_tx2
 
                  tx_energy_reader3 += calculate_energy(power, len(indicator_vector), (t_id + t_wait))
                  rx_energy_reader3 += calculate_energy(Preader_rx, sum(1 for x in indicator_vector if x == 1), (t_inf + t_wait))
                  wait_energy_reader3 += calculate_energy(Preader_wait, (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)), t_inf)
 
                  rx_energy_tag_sic3 += calculate_energy(Ptag_rx, len(indicator_vector), t_id)
                  tx_energy_tag_sic3 += calculate_energy(Ptag_tx, sum(1 for x in indicator_vector if x == 1), t_inf)


                  reader_time = calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf)
                  readers_sic_time3[reader_info[0]] += reader_time
                 

              # cập nhật tag chưa đọc của reader
                  unread_tags_current[reader_info[0]] = [t for t in reader_tags_current[reader_info[0]] if not t['status']]

      # Xử lý PIC cho tất cả reader song song     
      for reader_info in group:
         if unread_tags_current[reader_info[0]]:  # nếu vẫn còn tag chưa đọc
             pic_time = calculate_PIC_time(len(unread_tags_current[reader_info[0]]), t_inf, t_id)
             power = Preader_tx1 if reader_info[2] == READER_RADIUS else Preader_tx2

             tx_energy_reader3 += calculate_energy(power, len(unread_tags_current[reader_info[0]]), (t_id + t_wait))
             rx_energy_reader3 += calculate_energy(Preader_rx, len(unread_tags_current[reader_info[0]]), (t_inf + t_wait))

             tx_energy_tag_pic3 += calculate_energy(Ptag_tx, len(unread_tags_current[reader_info[0]]), t_inf)
             rx_energy_tag_pic3 += calculate_energy(Ptag_rx, len(unread_tags_current[reader_info[0]]) / 2, (t_id + t_inf))

             for tag in unread_tags_current[reader_info[0]]:
                 tag['status'] = True
             readers_pic_time_current_3[reader_info[0]] = pic_time
         else:
             readers_pic_time_current_3[reader_info[0]] = 0  # Nếu không còn tag, thời gian PIC là 0

      for reader_info in group:
          readers_time_3[reader_info[0]] = readers_sic_time3[reader_info[0]] + readers_pic_time_current_3[reader_info[0]]

      for reader_info in group:
         radius_other = reader_radius if reader_info[2] == reader_radius_small else reader_radius_small
         reader_tag = confirm_tags_in_range(reader_info[0], tags, reader_coords, radius_other)
         ungathered_tags = [t for t in reader_tag if not t['status']]
         pic_time = calculate_PIC_time(len(ungathered_tags), t_inf, t_id)
         power = Preader_tx1 if radius_other == reader_radius else Preader_tx2
         tx_energy_reader3 += calculate_energy(power, len(ungathered_tags), (t_id + t_wait))
         rx_energy_reader3 += calculate_energy(Preader_rx, len(ungathered_tags), (t_inf + t_wait))

         tx_energy_tag_pic3 += calculate_energy(Ptag_tx, len(ungathered_tags), t_inf)
         rx_energy_tag_pic3 += calculate_energy(Ptag_rx, len(ungathered_tags) / 2, (t_id + t_inf))

         for tag in ungathered_tags:
             tag['status'] = True
         readers_pic_time3[reader_info[0]] = pic_time


      group_time3 = max(readers_time_3) if readers_time_3 else 0
      for reader_info in group:
          group_time3 += readers_pic_time3[reader_info[0]]
      group_energy_reader3 += (tx_energy_reader3 + rx_energy_reader3 + wait_energy_reader3)
      group_energy_tag_sic3 += (tx_energy_tag_sic3 + rx_energy_tag_sic3)
      group_energy_tag_pic3 += (tx_energy_tag_pic3 + rx_energy_tag_pic3)

     # Cộng thời gian group này vào hệ thống
      system3_time += group_time3
      total_energy_reader3 += group_energy_reader3
      total_energy_tag_sic3 += group_energy_tag_sic3
      total_energy_tag_pic3 += group_energy_tag_pic3

  for tag in tags:  
     tag['status'] = False
     rd = None

  time3.append(system3_time)
  energy_reader3.append(total_energy_reader3)
  average_energy_tag3.append(1 / len(tags) * (total_energy_tag_sic3 + total_energy_tag_pic3))
    
 return time3, energy_reader3, average_energy_tag3

In [ ]:
def system4(activation_groups4, power_decisions, tags, reader_coords, reader_radius):
# Thời gian truyền của system 4
 system4_time = 0
 total_energy_reader4 = 0
 total_energy_tag_sic4 = 0
 total_energy_tag_pic4 = 0
 
 total_tags = 0

 emptySlots = [0] * len(reader_coords)
 collisionSlots = [0] * len(reader_coords)
 successTags = [0] * len(reader_coords)
 initTags = [0] * len(reader_coords)
 remainingTags = [0] * len(reader_coords)
 power = [0] * len(reader_coords)
 protocol = ['SIC'] * len(reader_coords)

 readers_time_4 = [0] * len(reader_coords)
 readers_pic_time4 = [0] * len(reader_coords)
 readers_sic_time4 = [0] * len(reader_coords)

 for group in sorted(activation_groups4, key=len, reverse=True):
     group_time4 = 0
    
     group_energy_reader4 = 0
     group_energy_tag_sic4 = 0
     group_energy_tag_pic4 = 0
     tx_energy_reader4 = 0
     rx_energy_reader4 = 0
     wait_energy_reader4 = 0

     tx_energy_tag_sic4 = 0
     rx_energy_tag_sic4 = 0
     tx_energy_tag_pic4 = 0
     rx_energy_tag_pic4 = 0

    # Vòng lặp round cho group
    # Chạy round đầu tiên cho tất cả reader trong group bằng SIC
     for reader_info in group:
         idx = reader_info[0]
         power[idx] = Preader_tx1 if power_decisions[idx] == 'High Power' else Preader_tx2
         reader_tags = [t for t in confirm_tags_in_range(idx, tags, reader_coords, reader_info[2]) if not t['status']]
         total_tags += len(reader_tags)
         initTags[idx] = len(reader_tags)
         remainingTags[idx] = len(reader_tags)
        
         rd = generate_unique()
         indicator_vector = create_indicator_vector(reader_tags, rd)
         transmissions = split_indicator_vector(indicator_vector)

         tx_energy_reader4 += calculate_energy(power[idx], len(indicator_vector), (t_id + t_wait))
         rx_energy_reader4 += calculate_energy(Preader_rx, sum(1 for x in indicator_vector if x == 1), (t_inf + t_wait))
         wait_energy_reader4 += calculate_energy(Preader_wait, (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)), t_inf)

         tx_energy_tag_sic4 += calculate_energy(Ptag_tx, sum(1 for x in indicator_vector if x == 1), t_inf)
         rx_energy_tag_sic4 += calculate_energy(Ptag_rx, len(indicator_vector), t_id)

         if (reader_tags):
             reader_time = calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf)
             readers_sic_time4[idx] += reader_time
         else:
             readers_sic_time4[idx] += 0

         successTags[idx] = sum(1 for x in indicator_vector if x == 1)
         emptySlots[idx] = sum(1 for x in indicator_vector if x == 0)
         collisionSlots[idx] = sum(1 for x in indicator_vector if x == -1)
         remainingTags[idx] = max(0, remainingTags[idx] - successTags[idx])

     group_reader_indices = [r[0] for r in group]
     while any(remainingTags[i] > 0 for i in group_reader_indices):
         for reader_info in group:
             idx = reader_info[0]
             ratio = (remainingTags[idx] / initTags[idx]) if initTags[idx] != 0 else 0
             protocol[idx] = 'PIC' if switchProtocol(
                 emptySlots[idx], ratio, collisionSlots[idx]
             ) == 'Switching' else 'SIC'

             if protocol[idx] == 'SIC':      
              rd = generate_unique()
              reader_tags = [t for t in confirm_tags_in_range(idx, tags, reader_coords, reader_info[2]) if t['status'] == False]
              indicator_vector = create_indicator_vector(reader_tags, rd)
              transmissions = split_indicator_vector(indicator_vector)

              tx_energy_reader4 += calculate_energy(power[idx], len(indicator_vector), (t_id + t_wait))
              rx_energy_reader4 += calculate_energy(Preader_rx, sum(1 for x in indicator_vector if x == 1), (t_inf + t_wait))
              wait_energy_reader4 += calculate_energy(Preader_wait, (len(indicator_vector) - sum(1 for x in indicator_vector if x == 1)), t_inf)

              rx_energy_tag_sic4 += calculate_energy(Ptag_rx, len(indicator_vector), t_id)
              tx_energy_tag_sic4 += calculate_energy(Ptag_tx, sum(1 for x in indicator_vector if x == 1), t_inf)

              if (remainingTags[idx]):
                 reader_time = calculate_SIC_time(transmissions, t_id, t_wait, indicator_vector, t_inf)
                 readers_sic_time4[idx] += reader_time
              else:
                 readers_sic_time4[idx] += 0
            
              successTags[idx] = sum(1 for x in indicator_vector if x == 1)
              emptySlots[idx] = sum(1 for x in indicator_vector if x == 0)
              collisionSlots[idx] = sum(1 for x in indicator_vector if x == -1)
              remainingTags[idx] = max(0, remainingTags[idx] - successTags[idx])

     # Xử lý PIC cho tất cả reader song song
             else:
              ungathered_tags = [t for t in confirm_tags_in_range(idx, tags, reader_coords, reader_info[2]) if not t['status']]
              pic_time = calculate_PIC_time(remainingTags[idx], t_inf, t_id)

              tx_energy_reader4 += calculate_energy(power[idx], remainingTags[idx], (t_id + t_wait))
              rx_energy_reader4 += calculate_energy(Preader_rx, remainingTags[idx], (t_inf + t_wait))

              tx_energy_tag_pic4 += calculate_energy(Ptag_tx, remainingTags[idx], t_inf)
              rx_energy_tag_pic4 += calculate_energy(Ptag_rx, remainingTags[idx] / 2, (t_id + t_inf))

              remainingTags[idx] = 0

              for tag in ungathered_tags:
               tag['status'] = True
              readers_pic_time4[idx] = pic_time

     for reader_info in group:
         idx = reader_info[0]
         readers_time_4[idx] = readers_sic_time4[idx] + readers_pic_time4[idx]

     group_time4 = max(readers_time_4[i] for i in group_reader_indices)
     group_energy_reader4 += (tx_energy_reader4 + rx_energy_reader4 + wait_energy_reader4)
     group_energy_tag_sic4 += (tx_energy_tag_sic4 + rx_energy_tag_sic4)
     group_energy_tag_pic4 += (tx_energy_tag_pic4 + rx_energy_tag_pic4)

    # Cộng thời gian group này vào hệ thống
     system4_time += group_time4
     total_energy_reader4 += group_energy_reader4
     total_energy_tag_sic4 += group_energy_tag_sic4
     total_energy_tag_pic4 += group_energy_tag_pic4
    
# --- Final sweep: vét cạn toàn bộ hệ (những tag chưa được bất kỳ group nào thu)
# Dùng biến cục bộ để tránh dùng lại tx_energy_reader4 v.v. từ trên
 final_tx_r = final_rx_r = 0
 final_tx_tag_pic = final_rx_tag_pic = 0
 final_total_tags_found = 0

 for ridx in range(len(reader_coords)):
     reader_tags = [t for t in confirm_tags_in_range(ridx, tags, reader_coords, reader_radius) if not t['status']]
     if not reader_tags:
         continue

    # số tag chưa đọc ở vùng này
     n = len(reader_tags)
     final_total_tags_found += n

     pic_time = calculate_PIC_time(n, t_inf, t_id)

     final_tx_r += calculate_energy(Preader_tx1, n, (t_id + t_wait))
     final_rx_r += calculate_energy(Preader_rx, n, (t_inf + t_wait))


     final_rx_tag_pic += calculate_energy(Ptag_rx, (n / 2), (t_id + t_inf))
     final_tx_tag_pic += calculate_energy(Ptag_tx, n, t_inf)

     for tag in reader_tags:
         tag['status'] = True

     readers_pic_time4[ridx] += pic_time
     system4_time += pic_time

# cộng vào tổng hệ thống
 total_energy_reader4 += (final_tx_r + final_rx_r)
 total_energy_tag_pic4 += (final_tx_tag_pic + final_rx_tag_pic)

 return system4_time, total_energy_reader4


#### Thời gian và năng lượng trung bình trên toàn threshold

In [ ]:
def average_system1(activation_groups1, tags, reader_coords, reader_radius):
    time1, energy_reader1, energy_tag1 = system1(activation_groups1, tags, reader_coords, reader_radius)
    
    a_time1 = sum(time1) / len(time1)
    a_energy_reader1 = sum(energy_reader1) / len(energy_reader1)
    a_energy_tag1 = sum(energy_tag1) / len(energy_tag1)

    return a_time1, a_energy_reader1, a_energy_tag1

In [ ]:
def average_system2(activation_groups2, tags, reader_coords, reader_radius, reader_radius_small):
    time2, energy_reader2, energy_tag2 = system2(activation_groups2, tags, reader_coords, reader_radius, reader_radius_small)
    
    a_time2 = sum(time2) / len(time2)
    a_energy_reader2 = sum(energy_reader2) / len(energy_reader2)
    a_energy_tag2 = sum(energy_tag2) / len(energy_tag2)

    return a_time2, a_energy_reader2, a_energy_tag2

In [ ]:
def average_system3(activation_groups3, tags, reader_coords, reader_radius, reader_radius_small):
    time3, energy_reader3, energy_tag3 = system3(activation_groups3, tags, reader_coords, reader_radius, reader_radius_small)
    
    a_time3 = sum(time3) / len(time3)
    a_energy_reader3 = sum(energy_reader3) / len(energy_reader3)
    a_energy_tag3 = sum(energy_tag3) / len(energy_tag3)

    return a_time3, a_energy_reader3, a_energy_tag3

## Đồ thị so sánh thời gian và năng lượng

In [ ]:
# time1, energy_reader1, average_energy_tag1 = system1(activation_groups1, tags, reader_coords, READER_RADIUS)
# time2, energy_reader2, average_energy_tag2 = system2(activation_groups2, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
# time3, energy_reader3, average_energy_tag3 = system3(activation_groups3, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
# time4, energy_reader4, energy_reader4 = system4(activation_groups4, power_decisions, tags, reader_coords, READER_RADIUS)

In [ ]:
# plt.figure(figsize=(10, 6))

# # Vẽ các đường
# plt.plot(threshold, [lower_bound_time] * len(threshold),
#          linestyle="--", color="red", linewidth=1.5, label="Lower Bound")

# plt.plot(threshold, [t for t in time1],
#          marker='o', markersize=4, linewidth=1.2, alpha=0.8, label="System 1")

# plt.plot(threshold, [t for t in time2],
#          marker='s', markersize=4, linewidth=1.2, alpha=0.8, label="System 2")

# plt.plot(threshold, [t for t in time3],
#          marker='^', markersize=4, linewidth=1.2, alpha=0.8, label="System 3")

# plt.plot(threshold, [system4_time] * len(threshold),
#          marker='d', markersize=4, linewidth=1.2, alpha=0.8, label="System 4")

# # Nhãn trục
# plt.xlabel("Threshold (%)", fontsize=12)
# plt.ylabel("Thời gian (s)", fontsize=12)
# plt.title("So sánh thời gian giữa 4 System với lower bound", fontsize=14, fontweight='bold')

# # Giới hạn và ticks cho trục X 
# plt.xticks(np.arange(min(threshold), max(threshold)+0.1, 0.1))

# # Lưới nhẹ
# plt.grid(True, linestyle="--", alpha=0.3)

# # Legend đẹp hơn
# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 6))

# # Vẽ các đường
# plt.plot(threshold, [p for p in energy_reader1],
#          marker='o', markersize=4, linewidth=1.2, alpha=0.8, label="System 1")

# plt.plot(threshold, [p for p in energy_reader2],
#          marker='s', markersize=4, linewidth=1.2, alpha=0.8, label="System 2")

# plt.plot(threshold, [p for p in energy_reader3],
#          marker='^', markersize=4, linewidth=1.2, alpha=0.8, label="System 3")

# plt.plot(threshold, [energy_reader4] * len(threshold),
#          marker='d', markersize=4, linewidth=1.2, alpha=0.8, label="System 4")

# # Nhãn trục
# plt.xlabel("Threshold (%)", fontsize=12)
# plt.ylabel("Năng lượng (J)", fontsize=12)
# plt.title("So sánh năng lượng reader tiêu thụ giữa 4 System", fontsize=14, fontweight='bold')

# # Giới hạn và ticks cho trục X 
# plt.xticks(np.arange(min(threshold), max(threshold)+0.1, 0.1))

# # Lưới nhẹ
# plt.grid(True, linestyle="--", alpha=0.3)

# # Legend đẹp hơn
# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 6))

# # Vẽ các đường
# plt.plot(threshold, [ae for ae in average_energy_tag1],
#          marker='o', markersize=4, linewidth=1.2, alpha=0.8, label="System 1")

# plt.plot(threshold, [ae for ae in average_energy_tag2],
#          marker='s', markersize=4, linewidth=1.2, alpha=0.8, label="System 2")

# plt.plot(threshold, [ae for ae in average_energy_tag3],
#          marker='^', markersize=4, linewidth=1.2, alpha=0.8, label="System 3")

# plt.plot(threshold, [average_energy_tag4] * len(threshold),
#          marker='d', markersize=4, linewidth=1.2, alpha=0.8, label="System 4")

# # Nhãn trục
# plt.xlabel("Threshold (%)", fontsize=12)
# plt.ylabel("Năng lượng (J)", fontsize=12)
# plt.title("So sánh năng lượng tag tiêu thụ trung bình của 1 tag giữa 4 System", fontsize=14, fontweight='bold')

# # Giới hạn và ticks cho trục X 
# plt.xticks(np.arange(min(threshold), max(threshold)+0.1, 0.1))

# # Lưới nhẹ
# plt.grid(True, linestyle="--", alpha=0.3)

# # Legend đẹp hơn
# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()

## Ảnh hưởng của mật độ tags

### Inhomogeneous PPP

In [ ]:
# lam_values = np.linspace(1, 15, 10)

In [ ]:
# time1 = []
# energy_reader1 = []
# average_energy_tag1 = []

# time2 = []
# energy_reader2 = []
# average_energy_tag2 = []

# time3 = []
# energy_reader3 = []
# average_energy_tag3 = []

# time4 = []
# energy_reader4 = []
# for lam in lam_values:
#     regions = create_regions(AREA_SIZE, 7, (lam*0.8, lam*1.2))
#     tags = create_tags_inhomo(regions)

#     tag_x = [tag['coords'][0] for tag in tags]
#     tag_y = [tag['coords'][1] for tag in tags]
#     tag_coords = list(zip(tag_x, tag_y))
    
#     t1, er1, aet1 = system1(activation_groups1, tags, reader_coords, READER_RADIUS) 
#     time1.append(t1)
#     energy_reader1.append(er1)
#     average_energy_tag1.append(aet1)

    
#     t2, er2, aet2 = system2(activation_groups2, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL) 
#     time2.append(t2)
#     energy_reader2.append(er2)
#     average_energy_tag2.append(aet2)

#     t3, er3, aet3 = system3(activation_groups3, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL) 
#     time3.append(t3)
#     energy_reader3.append(er3)
#     average_energy_tag3.append(aet3)

#     power_decisions = classificationPowerPhase(list(range(len(reader_coords))), tags, weight, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
#     activation_groups4 = construct_activation_groups_system4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)

#     t4, er4 = system4(activation_groups4, power_decisions, tags, reader_coords, READER_RADIUS)
#     time4.append(t4)
#     energy_reader4.append(er4)


In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(lam_values, time1, marker='o', label='System 1')
# plt.plot(lam_values, time2, marker='s', label='System 2')
# plt.plot(lam_values, time3, marker='^', label='System 3')
# plt.plot(lam_values, time4, marker='d', label='System 4')

# plt.xlabel('λ trung bình (tag / m²)')
# plt.ylabel('Thời gian truyền (s)')
# plt.title('Thời gian truyền của 4 System với Inhomogeneous PPP')
# plt.grid(True, linestyle="--", alpha=0.3)

# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()


In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(lam_values, energy_reader1, marker='o', label='System 1')
# plt.plot(lam_values, energy_reader2, marker='s', label='System 2')
# plt.plot(lam_values, energy_reader3, marker='^', label='System 3')
# plt.plot(lam_values, energy_reader4, marker='d', label='System 4')

# plt.xlabel('λ trung bình (tag / m²)')
# plt.ylabel('Năng lượng tiêu thụ (J)')
# plt.title('Năng lượng tiêu thụ của 4 System với Inhomogeneous PPP')
# plt.grid(True, linestyle="--", alpha=0.3)

# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()


### Homogeneous PPP

In [ ]:
# lam_values_homo = np.linspace(1, 15, 25)

In [ ]:
# time1_homo = []
# energy_reader1_homo = []
# average_energy_tag1_homo = []

# time2_homo = []
# energy_reader2_homo = []
# average_energy_tag2_homo = []

# time3_homo = []
# energy_reader3_homo = []
# average_energy_tag3_homo = []

# time4_homo = []
# energy_reader4_homo = []

# for lam in lam_values_homo:
#     tags = create_tags_homo(lam, AREA_SIZE)
#     tag_x = [tag['coords'][0] for tag in tags]
#     tag_y = [tag['coords'][1] for tag in tags]
#     tag_coords = list(zip(tag_x, tag_y))

#     power_decisions = classificationPowerPhase(list(range(len(reader_coords))), tags, weight, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
#     activation_groups4 = construct_activation_groups_system4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)

#     t1, er1, aet1 = system1(activation_groups1, tags, reader_coords, READER_RADIUS)
#     time1_homo.append(t1)
#     energy_reader1_homo.append(er1)
#     average_energy_tag1_homo.append(aet1)

#     t2, er2, aet2 = system2(activation_groups2, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
#     time2_homo.append(t2)
#     energy_reader2_homo.append(er2)
#     average_energy_tag2_homo.append(aet2)

#     t3, er3, aet3 = system3(activation_groups3, tags, reader_coords)
#     time3_homo.append(t3)
#     energy_reader3_homo.append(er3)
#     average_energy_tag3_homo.append(aet3)

#     t4, er4 = system4(activation_groups4, power_decisions, tags, reader_coords, READER_RADIUS)
#     time4_homo.append(t4)
#     energy_reader4_homo.append(er4)


In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(lam_values_homo, time1_homo, marker='o', label='System 1')
# plt.plot(lam_values_homo, time2_homo, marker='s', label='System 2')
# plt.plot(lam_values_homo, time3_homo, marker='^', label='System 3')
# plt.plot(lam_values_homo, time4_homo, marker='d', label='System 4')
# plt.xlabel('λ trung bình (tag / m²)')
# plt.ylabel('Time (s)')
# plt.title('Thời gian truyền của 4 system với homogeneous')
# plt.grid(True, linestyle="--", alpha=0.3)


# plt.legend(loc="best", fontsize=10, frameon=True)

# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.plot(lam_values_homo, energy_reader1_homo, marker='o', label='System 1')
# plt.plot(lam_values_homo, energy_reader2_homo, marker='s', label='System 2')
# plt.plot(lam_values_homo, energy_reader3_homo, marker='^', label='System 3')
# plt.plot(lam_values_homo, energy_reader4_homo, marker='d', label='System 4')
# plt.xlabel('λ trung bình (tag / m²)')
# plt.ylabel('Energy (J)')
# plt.title('Năng lượng tiêu thụ của 4 system với homogeneous PPP')
# plt.grid(True, linestyle="--", alpha=0.3)
# plt.legend(loc="best", fontsize=10, frameon=True)
# plt.tight_layout()
# plt.show()

## Ảnh hưởng của vị trí reader

In [ ]:
grid = create_grid(AREA_SIZE, 301)
result = result(READER_RADIUS_SMALL, READER_RADIUS, 0.5, AREA_SIZE, grid)

rs = []
ds = []

for i in range(len(result)):
    rs.append(result[i][0])
    ds.append(result[i][1])

In [ ]:
time1 = []
energy_reader1 = []
average_energy_tag1 = []
time2 = []
energy_reader2 = []
average_energy_tag2 = []
time3 = []
energy_reader3 = []
average_energy_tag3 = []
time4 = []
energy_reader4 = []


tags = create_tags_homo(10, AREA_SIZE)
tag_x = [tag['coords'][0] for tag in tags]
tag_y = [tag['coords'][1] for tag in tags]
tag_coords = list(zip(tag_x, tag_y))

for r, d in zip(rs, ds):
    reader_coords = generate_reader_coords_symmetry(float(d))
    for tag in tags:
        tag['status'] = False
    READER_RADIUS = float(r)

    power_decisions = classificationPowerPhase(list(range(len(reader_coords))), tags, weight, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
    activation_groups1 = construct_activate_groups_system1(reader_coords, len(reader_coords), READER_RADIUS)
    activation_groups2 = construct_activate_groups_system2(reader_coords, len(reader_coords), READER_RADIUS, READER_RADIUS_SMALL)
    activation_groups3 = construct_activate_groups_system3(reader_coords, len(reader_coords), READER_RADIUS, READER_RADIUS_SMALL)
    activation_groups4 = construct_activation_groups_system4(power_decisions, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)

    t1, er1, aet1 = average_system1(activation_groups1, tags, reader_coords, READER_RADIUS)
    time1.append(t1)
    energy_reader1.append(er1)
    average_energy_tag1.append(aet1)

    t2, er2, aet2 = system2(activation_groups2, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
    time2.append(t2)
    energy_reader2.append(er2)
    average_energy_tag2.append(aet2)
    
    t3, er3, aet3 = average_system3(activation_groups3, tags, reader_coords, READER_RADIUS, READER_RADIUS_SMALL)
    time3.append(t3)
    energy_reader3.append(er3)
    average_energy_tag3.append(aet3)

    t4, er4 = system4(activation_groups4, power_decisions, tags, reader_coords, READER_RADIUS)
    time4.append(t4)
    energy_reader4.append(er4)

In [ ]:
plt.figure(figsize=(10, 6))
for r, d in zip(rs, ds):
    reader_coords = generate_reader_coords_symmetry(float(d))
    x, y = zip(*reader_coords)
    plt.scatter(x, y, label=f'd={float(d):.3f} m -- r={float(r):.3f} m', alpha=0.6)
    
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.legend()
plt.title('Reader Coordinates for Different Distances from Corner (Radius Step = 0.5 m)')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ds, time1, marker='o', label='System 1')
plt.plot(ds, time2, marker='s', label='System 2')
plt.plot(ds, time3, marker='^', label='System 3')
plt.plot(ds, time4, marker='d', label='System 4')
plt.xlabel('Khoảng cách so với góc phòng (m)')
plt.ylabel('Time (s)')
plt.title('Thời gian truyền của 4 system với homogeneous')
plt.grid(True, linestyle="--", alpha=0.3)

plt.legend(loc="best", fontsize=10, frameon=True)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ds, energy_reader1, marker='o', label='System 1')
plt.plot(ds, energy_reader2, marker='s', label='System 2')
plt.plot(ds, energy_reader3, marker='^', label='System 3')
plt.plot(ds, energy_reader4, marker='d', label='System 4')
plt.xlabel('Khoảng cách so với góc phòng (m)')
plt.ylabel('Energy (J)')
plt.title('Năng lượng tiêu thụ của 4 system với homogeneous PPP')
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(loc="best", fontsize=10, frameon=True)
plt.tight_layout()
plt.show()